In [1]:
import pandas as pd
import os
import numpy as np
import numpy.ma as ma
import glob
from netCDF4 import Dataset,date2num,num2date
import numpy as np
import pandas as pd
import warnings
import sympy as sp
import math
warnings.filterwarnings('ignore')
import xlsxwriter

In [2]:
def make_formula_1_var(df_SC,var1):
    a1=df_SC['SC_SVM'].count()
    b1=df_SC[var1].sum()
    d1=df_SC['R_10p'].sum()
    a2=b1
    b2=(df_SC[var1]**2).sum()
    d2=(df_SC[var1]*df_SC['R_10p']).sum()
    x, y = sp.symbols('x, y')
    eq1 = sp.Eq(a1*x+b1*y, d1)             
    eq2 = sp.Eq(a2*x+b2*y, d2)
    ans = sp.solve((eq1, eq2), (x, y))
    a=round(math.exp(ans[x]),2)
    b=round(ans[y],2)
    return a,b
def make_formula_2_var(df_SC,var1,var2):
    a1=df_SC['SC_SVM'].count()
    b1=df_SC[var1].sum()
    c1=df_SC[var2].sum()
    d1=df_SC['R_10p'].sum()
    a2=b1
    b2=(df_SC[var1]**2).sum()
    c2=(df_SC[var1]*df_SC[var2]).sum()
    d2=(df_SC[var1]*df_SC['R_10p']).sum()
    a3=c1
    b3=c2
    c3=(df_SC[var2]**2).sum()
    d3=(df_SC[var2]*df_SC['R_10p']).sum()
    x, y, z = sp.symbols('x, y, z')
    eq1 = sp.Eq(a1*x+b1*y+c1*z, d1)             
    eq2 = sp.Eq(a2*x+b2*y+c2*z, d2)
    eq3 = sp.Eq(a3*x+b3*y+c3*z, d3)
    ans = sp.solve((eq1, eq2,eq3), (x, y, z))
    a=round(math.exp(ans[x]),2)
    b=round(ans[y],2)
    c=round(ans[z],2)
    return a,b,c
def make_formula_3_var(df_SC,var1,var2,var3):
    a1=df_SC['SC_SVM'].count()
    b1=df_SC[var1].sum()
    c1=df_SC[var2].sum()
    d1=df_SC[var3].sum()
    e1=df_SC['R_10p'].sum()
    a2=b1
    b2=(df_SC[var1]**2).sum()
    c2=(df_SC[var1]*df_SC[var2]).sum()
    d2=(df_SC[var1]*df_SC[var3]).sum()
    e2=(df_SC[var1]*df_SC['R_10p']).sum()
    a3=c1
    b3=c2
    c3=(df_SC[var2]**2).sum()
    d3=(df_SC[var2]*df_SC[var3]).sum()
    e3=(df_SC[var2]*df_SC['R_10p']).sum()
    a4=d1
    b4=d2
    c4=d3
    d4=(df_SC[var3]**2).sum()
    e4=(df_SC[var3]*df_SC['R_10p']).sum()
    x, y, z, t = sp.symbols('x, y, z, t')
    eq1 = sp.Eq(a1*x+b1*y+c1*z+d1*t, e1)             
    eq2 = sp.Eq(a2*x+b2*y+c2*z+d2*t, e2)
    eq3 = sp.Eq(a3*x+b3*y+c3*z+d3*t, e3)
    eq4 = sp.Eq(a4*x+b4*y+c4*z+d4*t, e4)
    ans = sp.solve((eq1, eq2, eq3, eq4), (x, y, z, t))
    a=round(math.exp(ans[x]),2)
    b=round(ans[y],2)
    c=round(ans[z],2)
    d=round(ans[t],2)
    return a,b,c,d

In [3]:
Case='General.xlsx' #Full stratiform, Squall line
df_Stratiform_Z_ZDR=pd.read_excel(pd.ExcelFile("E:/data/QPE/at_stations_PA6/"+Case),sheet_name='Stratiform_Z_ZDR')
df_Convection_Z_ZDR=pd.read_excel(pd.ExcelFile("E:/data/QPE/at_stations_PA6/"+Case),sheet_name='Convection_Z_ZDR')
df_Stratiform_KDP=pd.read_excel(pd.ExcelFile("E:/data/QPE/at_stations_PA6/"+Case),sheet_name='Stratiform_Z_ZDR_KDP')
df_Convection_KDP=pd.read_excel(pd.ExcelFile("E:/data/QPE/at_stations_PA6/"+Case),sheet_name='Convection_Z_ZDR_KDP')
df_Z_noSVM=pd.read_excel(pd.ExcelFile("E:/data/QPE/at_stations_PA6/"+Case),sheet_name='Z')
a_S,b_S,c_S=make_formula_2_var(df_Stratiform_Z_ZDR,var1='Z_1500m',var2='ZDR_1500m')
a_C,b_C,c_C=make_formula_2_var(df_Convection_Z_ZDR,var1='Z_1500m',var2='ZDR_1500m')
aZ_S,bZ_S=make_formula_1_var(df_Stratiform_Z_ZDR,var1='Z_1500m')
aZ_C,bZ_C=make_formula_1_var(df_Convection_Z_ZDR,var1='Z_1500m')
aKDP_S,bKDP_S=make_formula_1_var(df_Stratiform_KDP,var1='KDP_1500m')
aKDP_C,bKDP_C=make_formula_1_var(df_Convection_KDP,var1='KDP_1500m')
a_3S,b_3S,c_3S,d_3S=make_formula_3_var(df_Stratiform_KDP,var1='Z_1500m',var2='ZDR_1500m',var3='KDP_1500m')
a_3C,b_3C,c_3C,d_3C=make_formula_3_var(df_Convection_KDP,var1='Z_1500m',var2='ZDR_1500m',var3='KDP_1500m')
a_Z,b_Z=make_formula_1_var(df_Z_noSVM,var1='Z_1500m')

In [4]:
print('Công thức với 3 biến Z, ZDR và KDP trường hợp '+Case[:-5]+' :')
print("Hệ số trong công thức mưa Stratiform:")
print("a=",a_3S,"b=",b_3S,"c=",c_3S, "d=",d_3S)
print("Hệ số trong công thức mưa Convection:")
print("a=",a_3C,"b=",b_3C,"c=",c_3C,"d=",d_3C)

Công thức với 3 biến Z, ZDR và KDP trường hợp General :
Hệ số trong công thức mưa Stratiform:
a= 2.26 b= 0.21 c= -1.08 d= 0.20
Hệ số trong công thức mưa Convection:
a= 173.39 b= -0.16 c= -1.90 d= 0.59


In [5]:
print('Công thức với 2 biến Z và ZDR trường hợp '+Case[:-5]+' :')
print("Hệ số trong công thức mưa Stratiform:")
print("a=",a_S,"b=",b_S,"c=",c_S)
print("Hệ số trong công thức mưa Convection:")
print("a=",a_C,"b=",b_C,"c=",c_C)

Công thức với 2 biến Z và ZDR trường hợp General :
Hệ số trong công thức mưa Stratiform:
a= 1.25 b= 0.24 c= -0.98
Hệ số trong công thức mưa Convection:
a= 4.17 b= 0.23 c= -1.93


In [6]:
print('Công thức với 1 biến Z trường hợp '+Case[:-5]+' :')
print("Hệ số trong công thức mưa Stratiform:")
print("a=",aZ_S,"b=",bZ_S)
print("Hệ số trong công thức mưa Convection:")
print("a=",aZ_C,"b=",bZ_C)

Công thức với 1 biến Z trường hợp General :
Hệ số trong công thức mưa Stratiform:
a= 1.19 b= 0.20
Hệ số trong công thức mưa Convection:
a= 3.13 b= 0.18


In [7]:
print('Công thức với 1 biến KDP trường hợp '+Case[:-5]+' :')
print("Hệ số trong công thức mưa Stratiform:")
print("a=",aKDP_S,"b=",bKDP_S)
print("Hệ số trong công thức mưa Convection:")
print("a=",aKDP_C,"b=",bKDP_C)

Công thức với 1 biến KDP trường hợp General :
Hệ số trong công thức mưa Stratiform:
a= 7.71 b= 0.39
Hệ số trong công thức mưa Convection:
a= 20.83 b= 0.49


In [8]:
print('Công thức với 2 biến Z và ZDR trường hợp '+Case[:-5]+' :')
print("Hệ số trong công thức mưa chung Z không phân loại:")
print("a=",a_Z,"b=",b_Z)

Công thức với 2 biến Z và ZDR trường hợp General :
Hệ số trong công thức mưa chung Z không phân loại:
a= 1.06 b= 0.23
